In [102]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [109]:
# Load the dataset
df_encoded = pd.read_csv("real_estate_texas_500_2024.csv")  # Change to your actual dataset file

# Drop unnecessary columns (e.g., 'id' and text-based features)
df_encoded.drop(columns=["id", "text", "url"], errors="ignore", inplace=True)

# Identify categorical columns automatically
categorical_columns = df_encoded.select_dtypes(include=["object"]).columns.tolist()

# Convert categorical columns to numerical (One-Hot Encoding)
df_encoded = pd.get_dummies(df_encoded, columns=categorical_columns, drop_first=True)

# Now that all columns are numerical, we can safely fill missing values
df_encoded.fillna(df_encoded.median(numeric_only=True), inplace=True)

# Ensure numerical data types are correct
df_encoded = df_encoded.astype({col: "float64" for col in df_encoded.columns if col != "listPrice"})

# Check the data
print(df_encoded.info())
print(df_encoded.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   listPrice           501 non-null    float64
 1   baths               501 non-null    float64
 2   baths_full          501 non-null    float64
 3   baths_full_calc     501 non-null    float64
 4   beds                501 non-null    float64
 5   sqft                501 non-null    float64
 6   stories             501 non-null    float64
 7   year_built          501 non-null    float64
 8   sub_type_townhouse  501 non-null    float64
 9   type_farm           501 non-null    float64
 10  type_land           501 non-null    float64
 11  type_mobile         501 non-null    float64
 12  type_multi_family   501 non-null    float64
 13  type_single_family  501 non-null    float64
 14  type_townhomes      501 non-null    float64
dtypes: float64(15)
memory usage: 58.8 KB
None
   listPrice  b